In [25]:
import pandas as pd
import json
from pandas import json_normalize

# Step 1: Open and load the JSON file
with open("20230727_195816_hn_sharings.json", 'r') as file:
    data = json.load(file)

# Step 2: Flatten the 'Sources' part without the 'ChatgptSharing'
sources_df = json_normalize(data['Sources'], 
                            meta=['Type', 'ID', 'URL', 'Author', 'Points', 'AttachedURL', 'Title', 'StoryText', 'CreatedAt', 'CommentsTotalCount'],
                            meta_prefix='source_',  # Add a prefix to avoid conflicts
                            errors='ignore')

# # Step 3: Flatten the 'ChatgptSharing' part separately, excluding URL in meta
# gpt_sharing_df = json_normalize(data['Sources'], 
#                                 record_path=['ChatgptSharing'], 
#                                 meta=['Type', 'ID', 'Author', 'Points', 'AttachedURL', 'Title', 'StoryText', 'CreatedAt', 'CommentsTotalCount'],  # Exclude 'URL' from here
#                                 errors='ignore')

# # Step 4: Merge the two DataFrames (sources_df and gpt_sharing_df) on a common key (if exists)
# merged_df = pd.merge(sources_df, gpt_sharing_df, on='ID', how='outer', suffixes=('_source', '_gpt'))

# # Step 5: Display the resulting DataFrame
# print(merged_df)

# Step 6: Save to CSV
sources_df.to_csv("full_data.csv", index=False)
